# Ops-Style PDF Tool Demo

This notebook shows how to call the document tools directly like an ops analyst:
1. Load a PDF
2. Build document map (sections, definitions, xrefs)
3. Search for covenants
4. Read spans by anchor
5. Follow references
6. Save findings in scratchpad and reading trail


In [ ]:
import sys
from pathlib import Path

# Adjust this path if your notebook cwd is different
repo_root = Path('/Users/infantjerin/Projects/git/coding-agent-doc')
sys.path.insert(0, str(repo_root / 'src'))

from tools.document_tools import (
    LoadDocumentsTool,
    BuildDocMapTool,
    SearchInDocTool,
    ReadSpanTool,
    FollowReferenceTool,
    QuoteEvidenceTool,
    ConsistencyCheckTool,
)
from tools.state_tools import WriteScratchpadTool, ReadScratchpadTool, AppendReadingTrailTool


In [ ]:
# Put your real PDF path here
pdf_path = '/absolute/path/to/credit_agreement.pdf'
# Example fallback to sample txt in repo if no PDF yet:
# pdf_path = str(repo_root / 'examples' / 'sample_credit_agreement.txt')

loader = LoadDocumentsTool()
doc_store = loader.run([pdf_path])
doc_store


In [ ]:
mapper = BuildDocMapTool()
doc_map = mapper.run(doc_store)

summary = {
    'documents': len(doc_map['document_store']['documents']),
    'sections': len(doc_map['sections']),
    'definitions': len(doc_map['definitions']),
    'xrefs': len(doc_map['xrefs']),
}
summary


In [ ]:
search = SearchInDocTool()
hits = search.run(doc_map=doc_map, query='covenants', scope='section', top_k=5)
hits


In [ ]:
reader = ReadSpanTool()
trail = AppendReadingTrailTool()
state = {'scratchpad': {}, 'reading_trail': []}

first_anchor = hits[0]['anchor'] if hits else None
span = reader.run(doc_map=doc_map, anchor=first_anchor) if first_anchor else None
if first_anchor:
    trail.run(state=state, anchor=first_anchor)

span


In [ ]:
# Follow one reference from the first anchor (if available)
follower = FollowReferenceTool()
refs = [r for r in doc_map['xrefs'] if first_anchor and r.get('from_anchor') == first_anchor]
refs[:3]


In [ ]:
resolved = None
if refs:
    resolved = follower.run(doc_map=doc_map, ref_id=refs[0]['id'])
    if resolved.get('resolved'):
        trail.run(state=state, anchor=resolved['anchor'])
resolved


In [ ]:
writer = WriteScratchpadTool()
reader_pad = ReadScratchpadTool()

writer.run(state=state, key='covenants_check', content={
    'query': 'covenants',
    'top_hit': hits[0] if hits else None,
    'resolved_ref': resolved,
})

reader_pad.run(state=state, key='covenants_check')


In [ ]:
quote = QuoteEvidenceTool()
check = ConsistencyCheckTool()

anchors = list(dict.fromkeys(state['reading_trail']))
evidence = quote.run(doc_map=doc_map, anchors=anchors)
consistency = check.run(claim='What are the financial covenants?', evidence=evidence)

{'reading_trail': state['reading_trail'], 'evidence': evidence, 'consistency': consistency}
